In [1]:
def process_category(cat_st, count_th = 30):
    vc = cat_st.value_counts()
    mask = vc.values > count_th
    cat2id = dict(zip(vc.index[mask], range(sum(mask))))

    next_id = sum(mask)

    if(cat_st.isna().sum() > count_th):
        cat2id["nan_value"] = next_id 
        next_id += 1

    rare_count = vc.values[~mask].sum() + (not("nan_value" in cat2id))*(cat_st.isna().sum())

    if(rare_count > count_th):
        cat2id["rare_value"] = next_id
        
    return cat2id
    
from sklearn.preprocessing import QuantileTransformer
def process_continues(cont_st, count_th = 30):
    
    qt = QuantileTransformer()
    vals = cont_st[~cont_st.isna()]
    qt.fit(vals.values.reshape((-1, 1)))
    
    d = {"transformer":qt}
    if(cont_st.isna().sum() > count_th):
        d["nan_extra"] = True
    return d

def transfrom_category(cat_st, cat2id):
    rez = []
    for x in cat_st:
        if(pd.isnull(x)):#if(np.isnan(x)):
            if("nan_value" in cat2id):
                rez.append(cat2id["nan_value"])
            elif("rare_value" in cat2id):
                rez.append(cat2id["rare_value"])
            else:
                rez.append(0)
        else:
            if(x in cat2id):
                rez.append(cat2id[x])
            elif("rare_value" in cat2id):
                rez.append(cat2id["rare_value"])
            else:
                rez.append(0)
    return np.array(rez, dtype = np.int64)

def transfrom_continues(cont_st, d):
    rez = np.zeros((len(cont_st), ), dtype = np.float64)
    
    mask = ~cont_st.isna().values
    
    vals = cont_st[mask].values.reshape((-1, 1))
    
    rez[mask] = d["transformer"].transform(vals).flatten()
    
    if("nan_extra" in  d):
        rez[~mask] = 0
        
        return rez, ~mask
    else:
        rez[~mask] = 0.5
        return rez, None

In [2]:
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np
import pandas as pd
class DfToTabNetMatrix(BaseEstimator, TransformerMixin):
    
  def fit(self, df, y = None):
    
    self.cat_feat = df.select_dtypes(include=['category']).columns.values.tolist()
    self.num_feat = list(set(df.columns).difference(self.cat_feat))
    
    self.cat_feat_properties = {}
    self.num_feat_properties = {}
    
    
    for col in self.cat_feat:
        cat_st = df[col]
        cat2id = process_category(cat_st)
        self.cat_feat_properties[col] = cat2id
        
    for col in self.num_feat:
        cont_st = df[col] #.astype("float64")
        d = process_continues(cont_st)
        self.num_feat_properties[col] = d
    
    l = []
    for x in self.num_feat:
        if 'nan_extra' in self.num_feat_properties[x]:
            l.append(x)

    for x in self.num_feat:
        if not('nan_extra' in self.num_feat_properties[x]):
            l.append(x)
    
    self.num_feat = l[:]
        
    return self

  def transform(self, X, y = None):
      cat_processed = []
      for col in self.cat_feat:
        cat2id = self.cat_feat_properties[col]
        cat_processed.append(transfrom_category(X[col], cat2id))
      
      num_processed = []
      flags = []
      for col in self.num_feat:
        d = self.num_feat_properties[col]
        val, flag = transfrom_continues(X[col], d)
        if(flag is None):
            pass
        else:
            flags.append(flag)
        
        num_processed.append(val)
      
    #TODO return 
      return (np.vstack(cat_processed).T).astype(np.int64), \
             (np.vstack(num_processed).T).astype(np.float32), \
             (np.vstack(flags).T).astype(np.float32)

In [3]:
class StartClusterFeaturesForLast(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    def fit(self, train):
        return self

    def transform(self, df):
        #prev1 prev2 
        clusters = []
        ohs = []

        for _, x in df.groupby("id"):
            cc = x.start_cluster.tolist()


            clusters.append([cc[0], cc[1]])
            
        df_clust = pd.DataFrame(clusters, columns=["prev2_start_cluster", "prev_start_cluster"])
        
        return df_clust
    
class StartClusterFeaturesForLastСut(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    def fit(self, train):
        return self

    def transform(self, df):
        #prev1 prev2 
        clusters = []
        ohs = []

        for _, x in df.groupby("id"):
            cc = x.start_cluster.tolist()


            clusters.append([cc[0]])
            
        df_clust = pd.DataFrame(clusters, columns=["prev_start_cluster"])
        
        return df_clust

In [4]:
test = pd.read_parquet('test_data.pqt', engine='pyarrow')
full_users = set(test[test.date == "month_4"].id)
test_ful = test[test.id.isin(full_users)]
test_cut = test[~test.id.isin(full_users)]


In [5]:
import pandas as pd
train = pd.read_parquet('train_data.pqt', engine='pyarrow')

In [6]:
cluster_feat_ful = StartClusterFeaturesForLast().transform(train)
cluster_feat_cut = StartClusterFeaturesForLastСut().transform(train)

cluster_feat_test_ful = StartClusterFeaturesForLast().transform(test_ful)
cluster_feat_test_cut = StartClusterFeaturesForLastСut().transform(test_cut)
cluster_feat_test_ful2cutpretrain = StartClusterFeaturesForLastСut().transform(test_ful)


In [7]:
train_ful = pd.concat([train[train.date=="month_3"].copy().reset_index(), cluster_feat_ful], axis=1)
train_cut = pd.concat([train[train.date=="month_3"].copy().reset_index(), cluster_feat_cut], axis=1)

test_ful = pd.concat([test_ful[test_ful.date=="month_6"].copy().reset_index(), cluster_feat_test_ful], axis=1)
test_cut = pd.concat([test_cut[test_cut.date=="month_6"].copy().reset_index(), cluster_feat_test_cut], axis=1)

In [8]:
y_train = train[train.date=="month_3"].copy().reset_index()["end_cluster"].values

In [9]:
def mark_cat_feat(df):
    cat_feat = []
    for x, y in zip(df.dtypes.index, df.dtypes.values):
        if(y == "object"):
            cat_feat.append(x)

    for cf  in cat_feat:
        df[cf] = df[cf].astype('category')

In [10]:
def filter_df_columns(df):
    features = []
    for f in df.columns.values:
        if not (f in ["id", "date", "end_cluster", "start_cluster", "index"]):
            features.append(f)
    return df[features]

train_cut = filter_df_columns(train_cut)
train_ful = filter_df_columns(train_ful)
test_ful = filter_df_columns(test_ful)
test_cut = filter_df_columns(test_cut)

mark_cat_feat(train_cut)
mark_cat_feat(train_ful)
mark_cat_feat(test_ful)
mark_cat_feat(test_cut)

In [11]:
pretrain_dataset = pd.concat([train_ful, test_ful])

In [12]:
mark_cat_feat(pretrain_dataset)

In [13]:
from tqdm.notebook import tqdm

def cast(x):
    if(isinstance(x, float)):
        if np.isnan(x):
            return "nan"
        else:
            return x #str(x)
    else:
        return x

class DatasetCorruptor(BaseEstimator, TransformerMixin):
    def __init__(self, rate=0.1):
        self.rate = rate
    
    def fit(self, df):
        self.df = df.copy()

        self.all_cols = df.columns.values

        self.d_int = {}


        self.change_stat = {}
        for col in tqdm(self.all_cols):
            val2id = {x:e for e, x in enumerate(set([cast(x) for x in df[col]]))}
            id2val = {v:k for k, v in val2id.items()}

            self.change_stat[col] = {"val2id":val2id, "id2val":id2val}

            self.d_int[col] = np.array([val2id[cast(x)] for x in df[col]])
        
        return self

    def transform(self, X=None, y=None):
        
        mutated = self.df.copy()

        mask_cols = []

        for col in self.all_cols:
            mask = np.random.binomial(1, self.rate, len(self.d_int[col])) == 1

            mask_cols.append(mask)

            new_vals = np.random.randint(low = 0, high = len(self.change_stat[col]["id2val"]) - 1, size = mask.sum())
            mask_increase = new_vals >= self.d_int[col][mask]
            new_vals[mask_increase] += 1

            new_vals = [self.change_stat[col]["id2val"][x] if self.change_stat[col]["id2val"][x] != "nan" else np.nan for x in new_vals]
            mutated.loc[mask, col] = new_vals
        
        
        return mutated, (np.stack(mask_cols).T).astype(np.float32)
        

In [14]:
#dc = DatasetCorruptor()

In [15]:
#dc.fit(train_ful)

In [16]:
#dc.transform(None)

In [17]:
import torch
class BaseDataset(torch.utils.data.Dataset):
    def __init__(self, cat_features, num_features, flags, y, y_map):
        self.a = cat_features
        self.b = num_features
        self.c = flags
        
        self.y = y
        self.y_map = y_map
        
        
    def __len__(self):
        return self.a.shape[0]
    
    def __getitem__(self, idx):
        return self.a[idx], self.b[idx], self.c[idx], self.y_map[self.y[idx]] 
    
class PretrainDataset(torch.utils.data.Dataset):
    def __init__(self, cat_features, num_features, flags, y):
        self.a = cat_features
        self.b = num_features
        self.c = flags
        
        self.y = y
        
        
    def __len__(self):
        return self.a.shape[0]
    
    def __getitem__(self, idx):
        return self.a[idx], self.b[idx], self.c[idx], self.y[idx]

import torch
class TestDataset(torch.utils.data.Dataset):
    def __init__(self, cat_features, num_features, flags):
        self.a = cat_features
        self.b = num_features
        self.c = flags
        
    def __len__(self):
        return self.a.shape[0]
    
    def __getitem__(self, idx):
        return self.a[idx], self.b[idx], self.c[idx] 


cluster_weights = pd.read_excel("./cluster_weights.xlsx").set_index("cluster")
weights_dict = cluster_weights["unnorm_weight"].to_dict()

order = list(weights_dict.keys())
label2pos = {x:e for e, x in enumerate(order)}

batch_size = 256


In [18]:
import torch.nn.functional as F
import torch.nn as nn

from torch.nn.parameter import Parameter
from torch import Tensor
import math
    
import torch.optim as optim
import transformers

class PeriodicEmbeddings(nn.Module):
    # Source: https://github.com/Yura52/tabular-dl-num-embeddings/blob/e49e95c52f829ad0ab7d653e0776c2a84c03e261/lib/deep.py#L28
    def __init__(self, n_features: int, d_embedding: int, sigma: float) -> None:
        if d_embedding % 2:
            raise ValueError('d_embedding must be even')

        super().__init__()
        self.sigma = sigma
        self.coefficients = Parameter(Tensor(n_features, d_embedding // 2))
        self.reset_parameters()

    def reset_parameters(self):
        nn.init.normal_(self.coefficients, 0.0, self.sigma)

    def forward(self, x):
        if x.ndim != 2:
            raise ValueError('The input must have two dimensions')
        x = 2 * math.pi * self.coefficients[None] * x[..., None]
        return torch.cat([torch.cos(x), torch.sin(x)], -1)
    

class NLinear(nn.Module):

    def __init__(self, n_tokens, d_in, d_out, bias = True):
        super().__init__()
        self.weight = Parameter(Tensor(n_tokens, d_in, d_out))
        self.bias = Parameter(Tensor(n_tokens, d_out)) if bias else None
        self.reset_parameters()

    def reset_parameters(self):
        # This initialization is equivalent to that of torch.nn.Linear
        d_in = self.weight.shape[1]
        bound = 1 / math.sqrt(d_in)
        nn.init.uniform_(self.weight, -bound, bound)
        if self.bias is not None:
            nn.init.uniform_(self.bias, -bound, bound)

    def forward(self, x):
        if x.ndim != 3:
            raise ValueError(
                'The input must have three dimensions (batch_size, n_tokens, d_embedding)'
            )
        x = x[..., None] * self.weight[None]
        x = x.sum(-2)
        if self.bias is not None:
            x = x + self.bias[None]
        return x
    
def make_plr_embeddings(
    n_features: int, d_embedding: int, d_periodic_embedding: int, sigma: float
) -> nn.Module:
    return nn.Sequential(
        PeriodicEmbeddings(n_features, d_periodic_embedding, sigma),
        NLinear(n_features, d_periodic_embedding, d_embedding),
        nn.ReLU(),
    )

class Block(nn.Module):
        """The main building block of `MLP`."""

        def __init__(
            self,
            d_in,
            d_out,
            bias,
            activation,
            dropout,
        ) -> None:
            super().__init__()
            self.linear = nn.Linear(d_in, d_out, bias)
            self.activation = activation
            self.dropout = nn.Dropout(dropout)

        def forward(self, x):
            return self.dropout(self.activation(self.linear(x)))



def calculate_embedding_size_by_cardinality(card):
    return int(1.6*card**0.5)

class Dense(torch.nn.Module):
    def __init__(self, obj, num_feat, num_fourier = 20, eps=2):
        super().__init__()
        
        self.num_feat = num_feat
        
        d = obj.cat_feat_properties
        
        self.embedding_layers = nn.ModuleList([nn.Embedding(len(d[k]), calculate_embedding_size_by_cardinality(len(d[k]))) for k in obj.cat_feat])
    
        self.cat_embeds_sizes = []
        num_neyrons_first_level = 0
        for k in obj.cat_feat:
            num = calculate_embedding_size_by_cardinality(len(d[k]))
            num_neyrons_first_level += num
            self.cat_embeds_sizes.append(num)
        
        self.num_of_cats = len(obj.cat_feat)
        
        
        d = obj.num_feat_properties
        for k in obj.num_feat:
            if('nan_extra' in d[k]):
                num_neyrons_first_level += 1
        
        self.periodic_embeds_layer =make_plr_embeddings( len(obj.num_feat), 2*num_fourier, 2*num_fourier, eps) # PeriodicEmbedding(len(obj.num_feat), num_fourier, sigma = 30)
        
        
        self.n_fourier = num_fourier
        self.num_of_nums = len(obj.num_feat)
        
        num_neyrons_first_level += 2*self.n_fourier*self.num_of_nums
        
        d_layers = [num_neyrons_first_level, 512] #512
        
        dropout = 0.2
        self.inner1 = nn.Linear(512, 200) 
        self.head1 = nn.Linear(200, len(order))
        
        self.dp = nn.Dropout(dropout)
        
        self.inner2 = nn.Linear(512, 512) 
        self.head2 = nn.Linear(512, self.num_feat)
        
        
        
        self.sm = nn.LogSoftmax(dim=1)
        
        
        self.body = nn.Sequential(
            *[
                Block(
                    d_in=d_layers[i],
                    d_out=d,
                    bias=True,
                    activation=nn.functional.relu,
                    dropout=dropout,
                )
                for i, d in enumerate(d_layers[1:])
            ]
        )
        
        print("first layer size", num_neyrons_first_level)
    
    def forward(self, cats, nums, flags):
        
        rezults = []
        for i in range(self.num_of_cats):
            x = cats[:, i]
            x = self.embedding_layers[i](x)
            rezults.append(x)
        
        cat_inp = torch.cat(rezults, axis=1)
        num_inp = self.periodic_embeds_layer(nums)
        num_inp = num_inp.reshape(-1, 2*self.n_fourier*self.num_of_nums)
        
        x = torch.cat([cat_inp, num_inp, flags], axis=1)
        
        x = self.body(x)
        
        out1 = self.head1(nn.functional.relu(self.inner1(x)))
        
        out2 = self.head2(self.dp(nn.functional.relu(self.inner2(x))))
        
        
        return self.sm(out1), out2


In [19]:
from sklearn.metrics import roc_auc_score
def weighted_roc_auc(y_true, y_pred, labels, weights_dict):
    unnorm_weights = np.array([weights_dict[label] for label in labels])
    weights = unnorm_weights / unnorm_weights.sum()
    classes_roc_auc = roc_auc_score(y_true, y_pred, #labels=labels,
                                    multi_class="ovr", average=None)
    return sum(weights * classes_roc_auc)

def evaluate():
    net.eval()

    outs = []
    rights = []

    for x in tqdm(test_loader):
        for i in range(len(x)):
            x[i] = x[i].to(device)

        out,_  = net(x[0], x[1], x[2])

        outs.append(out.detach().cpu().numpy())

        rights.append(x[3].detach().cpu().numpy())

    rights = np.hstack(rights)
    outs = np.vstack(outs)
    
    dif = set(np.arange(len(order))) - set(rights)
    
    if(len(dif)>0):
        print(dif)
        for e, x in enumerate(dif):
            rights[e] = x
    
    preds = sp.special.softmax(outs, axis=1)
    eval_score = weighted_roc_auc(rights, preds, order, weights_dict)

    print(eval_score)
    return eval_score, preds

def predict():
    net.eval()

    outs = []
    
    for x in tqdm(final_loader):
        for i in range(len(x)):
            x[i] = x[i].to(device)

        out,_  = net(x[0], x[1], x[2])

        outs.append(out.detach().cpu().numpy())

    outs = np.vstack(outs)
    
    return sp.special.softmax(outs, axis=1)
    

In [20]:
def epoch_steps(df, batch_size):
    if(len(df)%batch_size!=0):
        return len(df)//batch_size + 1
    else:
        return len(df)//batch_size
    
def sigmoid(x):
  return 1 / (1 + np.exp(-x))

In [21]:
import os

In [22]:
feat_trans = DfToTabNetMatrix()
    
feat_trans.fit(pretrain_dataset)

DfToTabNetMatrix()

In [31]:
from sklearn.model_selection import KFold, StratifiedKFold
import scipy as sp
from tqdm.notebook import tqdm
import copy
import shutil

run_dir = "v3run4"

y_train_id = [label2pos[x] for x in y_train]

if os.path.isdir(run_dir):
    shutil.rmtree(run_dir)


os.mkdir(run_dir)

cv_scores = []

for iteration in range(10):
    iter_dir = str(iteration)
    work_dir = os.path.join(run_dir, iter_dir) 
    os.mkdir(work_dir)

    kf = StratifiedKFold(n_splits=5, shuffle=True)

    eval_scores = []
    test_preds = []
    eval_preds = []


    for i, (train_index, test_index) in enumerate(kf.split(train_ful, y_train)):
        Xtr = train_ful.iloc[train_index]
        Xval = train_ful.iloc[test_index]

        ytr = y_train[train_index]
        yval = y_train[test_index]

        #feat_trans = DfToTabNetMatrix()

        train_a, train_b, train_c = feat_trans.transform(Xtr)
        val_a, val_b, val_c = feat_trans.transform(Xval)
        test_a, test_b, test_c = feat_trans.transform(test_ful)


        train_dataset = BaseDataset(train_a, train_b, train_c, ytr, label2pos)
        test_dataset = BaseDataset(val_a, val_b, val_c, yval, label2pos)
        final_dataset = TestDataset(test_a, test_b, test_c)

        train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=batch_size, shuffle=True,
        )

        test_loader = torch.utils.data.DataLoader(
        test_dataset, batch_size=batch_size, shuffle=False,
        )

        final_loader = torch.utils.data.DataLoader(
        final_dataset, batch_size=batch_size, shuffle=False,
        )

        net = Dense(feat_trans, len(pretrain_dataset.columns))
        #net.load_state_dict(torch.load(os.path.join("emodels1", str(19)+".pt")))

        device = "cuda"
        net.to(device)
        loss_function = torch.nn.NLLLoss() #weight=torch.tensor([weights_dict[x] for x in order]).float().to(device)

        epoches = 10

        optimizer = optim.Adam(net.parameters(), lr=5e-4)

        scheduler = transformers.get_linear_schedule_with_warmup(optimizer, int(0.1 * epoches * len(train_loader)), epoches * len(train_loader), last_epoch=-1)

        best_score = 0.0
        best_weights = net.state_dict()

        net.train()
        for _ in range(epoches):
            for x in tqdm(train_loader):
                for i in range(len(x)):
                    x[i] = x[i].to(device)

                out, _ = net(x[0], x[1], x[2])

                loss = loss_function(out, x[3])

                loss.backward(),
                optimizer.step(),
                optimizer.zero_grad()
                scheduler.step()

            rez, _ = evaluate()
            if(rez > best_score):
                best_score = rez
                best_weights = copy.deepcopy(net.state_dict())

            net.train()

        net.load_state_dict(best_weights)
        rez, preds = evaluate()
        eval_scores.append(rez)
        test_preds.append(predict())
        eval_preds.append((test_index, preds))


    train_preds = np.zeros((len(train_ful), len(order)))
    for inds, preds in eval_preds:
        for i, p in zip(inds, preds):
            train_preds[i] = p

    test_df_pred = pd.DataFrame(np.stack(test_preds).mean(axis=0), columns=order)
    train_df_pred = pd.DataFrame(train_preds, columns=order)
    cross_val_rez = np.mean(eval_scores)
    cv_scores.append(cross_val_rez)

    with open(os.path.join(work_dir, "cross_val_rez.txt"), "w") as f:
        print(cross_val_rez, file=f)
        print(weighted_roc_auc(y_train_id, train_preds, order, weights_dict), file=f)
        print(eval_scores, file=f)

    test_df_pred.to_csv(os.path.join(work_dir, "test.csv"))
    train_df_pred.to_csv(os.path.join(work_dir, "train.csv"))


first layer size 3412


  0%|          | 0/625 [00:00<?, ?it/s]

  0%|          | 0/157 [00:00<?, ?it/s]

0.8727301502352094


  0%|          | 0/625 [00:00<?, ?it/s]

...

  0%|          | 0/157 [00:00<?, ?it/s]

0.9139935188005133


  0%|          | 0/625 [00:00<?, ?it/s]

  0%|          | 0/157 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [22]:
pretrain_dataset = pd.concat([train_cut, test_cut, cluster_feat_test_ful2cutpretrain])
mark_cat_feat(pretrain_dataset)

feat_trans = DfToTabNetMatrix()
    
feat_trans.fit(pretrain_dataset)


DfToTabNetMatrix()

In [25]:
from sklearn.model_selection import KFold, StratifiedKFold
import scipy as sp
from tqdm.notebook import tqdm
import copy

y_train_id = [label2pos[x] for x in y_train]

run_dir = "run3"

os.mkdir(run_dir)

cv_scores = []

for iteration in range(10):
    
    iter_dir = str(iteration)
    work_dir = os.path.join(run_dir, iter_dir) 
    os.mkdir(work_dir)

    kf = StratifiedKFold(n_splits=5, shuffle=True)

    eval_scores = []
    test_preds = []
    eval_preds = []

    for i, (train_index, test_index) in enumerate(kf.split(train_cut, y_train)):
        Xtr = train_cut.iloc[train_index]
        Xval = train_cut.iloc[test_index]

        ytr = y_train[train_index]
        yval = y_train[test_index]

        #feat_trans = DfToTabNetMatrix()

        train_a, train_b, train_c = feat_trans.transform(Xtr)
        val_a, val_b, val_c = feat_trans.transform(Xval)
        test_a, test_b, test_c = feat_trans.transform(test_cut)


        train_dataset = BaseDataset(train_a, train_b, train_c, ytr, label2pos)
        test_dataset = BaseDataset(val_a, val_b, val_c, yval, label2pos)
        final_dataset = TestDataset(test_a, test_b, test_c)

        train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=batch_size, shuffle=True,
        )

        test_loader = torch.utils.data.DataLoader(
        test_dataset, batch_size=batch_size, shuffle=False,
        )

        final_loader = torch.utils.data.DataLoader(
        final_dataset, batch_size=batch_size, shuffle=False,
        )

        net = Dense(feat_trans, len(pretrain_dataset.columns))
        net.load_state_dict(torch.load(os.path.join("models3", str(149)+".pt")))

        device = "cuda"
        net.to(device)
        loss_function = torch.nn.NLLLoss() #weight=torch.tensor([weights_dict[x] for x in order]).float().to(device)

        epoches = 10

        optimizer = optim.AdamW(net.parameters(), lr=5e-4)

        scheduler = transformers.get_linear_schedule_with_warmup(optimizer, int(0.1 * epoches * len(train_loader)), epoches * len(train_loader), last_epoch=-1)

        best_score = 0.0
        best_weights = net.state_dict()

        net.train()
        for _ in range(epoches):
            for x in tqdm(train_loader):
                for i in range(len(x)):
                    x[i] = x[i].to(device)

                out, _ = net(x[0], x[1], x[2])

                loss = loss_function(out, x[3])

                loss.backward(),
                optimizer.step(),
                optimizer.zero_grad()
                scheduler.step()

            rez, _ = evaluate()
            if(rez > best_score):
                best_score = rez
                best_weights = copy.deepcopy(net.state_dict())

            net.train()

        net.load_state_dict(best_weights)
        rez, preds = evaluate()
        eval_scores.append(rez)
        test_preds.append(predict())
        eval_preds.append((test_index, preds))
        
        
    train_preds = np.zeros((len(train_ful), len(order)))
    for inds, preds in eval_preds:
        for i, p in zip(inds, preds):
            train_preds[i] = p

    test_df_pred = pd.DataFrame(np.stack(test_preds).mean(axis=0), columns=order)
    train_df_pred = pd.DataFrame(train_preds, columns=order)
    cross_val_rez = np.mean(eval_scores)
    cv_scores.append(cross_val_rez)

    with open(os.path.join(work_dir, "cross_val_rez.txt"), "w") as f:
        print(cross_val_rez, file=f)
        print(weighted_roc_auc(y_train_id, train_preds, order, weights_dict), file=f)
        print(eval_scores, file=f)

    test_df_pred.to_csv(os.path.join(work_dir, "test.csv"))
    train_df_pred.to_csv(os.path.join(work_dir, "train.csv"))



first layer size 3675


  0%|          | 0/625 [00:00<?, ?it/s]

  0%|          | 0/157 [00:00<?, ?it/s]

0.8551596749074653


  0%|          | 0/625 [00:00<?, ?it/s]

...

  0%|          | 0/157 [00:00<?, ?it/s]

0.9028175570472576


  0%|          | 0/39 [00:00<?, ?it/s]

first layer size 3675


  0%|          | 0/625 [00:00<?, ?it/s]

In [2]:
import pandas as pd

In [28]:
test = pd.read_parquet('test_data.pqt', engine='pyarrow')
full_users = set(test[test.date == "month_4"].id)
test_ful = test[test.id.isin(full_users)]
test_cut = test[~test.id.isin(full_users)]

test_ful_preds = pd.read_csv("v2run1/0/test.csv").drop(columns=["Unnamed: 0"])
test_ful_preds["id"] = test_ful[test_ful.date=="month_4"].id.values

test_cut_preds = pd.read_csv("consolidated_test_run2.csv").drop(columns=["Unnamed: 0"])
test_cut_preds["id"] = test_cut[test_cut.date=="month_6"].id.values


In [29]:
concated = pd.concat([test_ful_preds, test_cut_preds])

In [30]:
concated  = concated.set_index("id").reset_index()

In [31]:
concated = concated.sort_values("id").reset_index().drop(columns=["index"])